# 0. Environment Setup with pip

This command installs all required libraries in one step for a macOS-based development environment. It includes deep learning (Keras/TensorFlow), machine learning (XGBoost), hyperparameter optimization (Hyperopt, Optuna), experiment tracking (MLflow), and model wrapping (Scikeras).

In [3]:
!pip install -q --disable-pip-version-check \
    keras \
    tensorflow-macos \
    tensorflow-metal \
    xgboost \
    hyperopt \
    mlflow \
    optuna \
    scikeras \
    gdown

# 1. Import libraries

This section imports all necessary Python libraries and modules required for data handling, model building, evaluation, optimization, and visualization.

In [ ]:
# --- Standard Library ---
import os
from pathlib import Path

# --- Progress Tracking ---
from tqdm import tqdm

# --- Data Handling ---
import numpy as np
import pandas as pd

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Scikit-learn ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --- Model Persistence ---
import joblib

# --- MLflow for Experiment Tracking ---
import mlflow
import mlflow.sklearn
import mlflow.keras
from mlflow.models.signature import infer_signature

# --- XGBoost ---
import xgboost as xgb

# --- Keras / TensorFlow ---
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# --- SciKeras (optional Keras wrapper for sklearn) ---
from scikeras.wrappers import KerasRegressor

# --- Hyperparameter Optimization ---
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK


This snippet ensures the script can access project-level configuration regardless of execution context. It dynamically sets the project root, adjusts the system path for imports, and loads configuration constants needed for forecasting workflows.

In [5]:
import sys
import os

# Get project root one level up from current working directory (for Jupyter use)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add project root to Python path if not already included
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import constants from Streamlit app configuration
import importlib
import app.config as cfg
importlib.reload(cfg)

import model.model_utils as model_utils



def main():
    print("Features:", cfg.FEATURES)
    print("Sequence Length:", cfg.SEQ_LEN)
    print("Target:", cfg.TARGET)
    print("Cutoff Date:", cfg.CUTOFF_DATE)
    print("Forecast End:", cfg.FORECAST_END)
    print("Hyperopt Space:", cfg.HYPEROPT_SPACE)

if __name__ == "__main__":
    main()

Features: ['store_nbr', 'item_nbr', 'onpromotion', 'day_of_week', 'month', 'unit_sales_7d_avg', 'lag_1', 'lag_7', 'rolling_mean_7']
Sequence Length: 90
Target: unit_sales
Cutoff Date: 2013-12-31 00:00:00
Forecast End: 2014-03-31
Hyperopt Space: {'max_depth': [3, 4, 5, 6], 'learning_rate_range': (0.01, 0.1, 0.2, 0.3), 'n_estimators': [20, 50, 100]}


# 2. Import Data

Import data prepared in the first notebook.

In [6]:
df_train = pd.read_csv("../data/preprocessed_data/train_guayas_prepared.csv", parse_dates=['date'])

# 3. Data Preparation

## 3.1 Create feature and set dtypes

This section performs essential preprocessing tasks, such as datetime conversion, label encoding, memory optimization, and feature engineering. It includes lag features and rolling statistics, which are commonly used in time series forecasting models.

In [7]:
# 1. Convert the 'date' column to datetime
df_train['date'] = pd.to_datetime(df_train['date'])

# 2. Label-encode family on train, then apply same encoder to test
le = LabelEncoder()
df_train['family_code'] = le.fit_transform(df_train['family'])
df_train ['family_code'] = le.transform   (df_train ['family']).astype('uint8')

# 3. Downcast integer columns to reduce memory usage
df_train['store_nbr']   = df_train['store_nbr'].astype('uint8')
df_train['item_nbr']    = df_train['item_nbr'].astype('uint32')
df_train['day_of_week'] = df_train['day_of_week'].astype('uint8')
df_train['month']       = df_train['month'].astype('uint8')
df_train['year']        = df_train['year'].astype('uint16')
df_train['onpromotion'] = df_train['onpromotion'].astype('bool')

# 4. Downcast other numeric columns where possible
df_train['unit_sales']       = pd.to_numeric(df_train['unit_sales'],       downcast='float')
df_train['unit_sales_7d_avg'] = pd.to_numeric(df_train['unit_sales_7d_avg'], downcast='float')

# 5. Lag features
df_train['lag_1'] = df_train['unit_sales'].shift(1)
df_train['lag_7'] = df_train['unit_sales'].shift(7)

# 6. Rolling-Window-Features
df_train['rolling_mean_7']     = df_train['unit_sales'].shift(1).rolling(window=7).mean()
df_train['unit_sales_7d_avg']  = df_train['unit_sales'].shift(1).rolling(window=7).mean()

# 7. Drop rows with NaN values
df_train = df_train.dropna(subset=['lag_1', 'lag_7', 'rolling_mean_7']).reset_index(drop=True)

# 8. Drop unnecessary columns
int_cols = df_train.select_dtypes(include='int').columns
df_train[int_cols] = df_train[int_cols].astype("float16")

print(df_train.filter(['date','unit_sales','lag_1','lag_7','rolling_mean_7']).head(10))

        date  unit_sales  lag_1  lag_7  rolling_mean_7
0 2013-01-09         2.0    0.0    0.0        0.000000
1 2013-01-10         0.0    2.0    0.0        0.285714
2 2013-01-11         0.0    0.0    0.0        0.285714
3 2013-01-12         2.0    0.0    0.0        0.285714
4 2013-01-13         0.0    2.0    0.0        0.571429
5 2013-01-14         0.0    0.0    0.0        0.571429
6 2013-01-15         0.0    0.0    0.0        0.571429
7 2013-01-16         1.0    0.0    2.0        0.571429
8 2013-01-17         2.0    1.0    0.0        0.428571
9 2013-01-18         0.0    2.0    0.0        0.714286


## 3.2 Save the dataset

After the dataset is prepared and further features were included, the dataset has to be saved.

In [8]:
df_train.to_csv("../data/preprocessed_data/train_guayas_model_ready.csv", index=False)

## 3.3 Filter for best Store-Item-combination

This section filters the dataset to include only data from 2013 and computes basic statistics per store and item combination. It selects only combinations with sufficient data coverage and calculates a custom score based on mean and standard deviation of sales. The best-performing combination is selected to serve as the focus for subsequent model training.

In [9]:
# Filter 2013 data only
df_2013 = df_train[df_train["date"].dt.year == 2013]

# Group by store and item to compute statistics
stats = df_2013.groupby(["store_nbr", "item_nbr"]).agg(
    count_days=("unit_sales", "count"),
    mean_sales=("unit_sales", "mean"),
    std_sales=("unit_sales", "std"),
    positive_days=("unit_sales", lambda x: (x > 0).sum())
).reset_index()

# Filter only combinations with sufficient data
stats = stats[stats["count_days"] >= 300]

# Compute a custom score: mean / std
stats["score"] = stats["mean_sales"] / (stats["std_sales"] + 1e-5)

# Get the best combination
best_combo = stats.sort_values("score", ascending=False).iloc[0]
store_id = int(best_combo["store_nbr"])
item_id = int(best_combo["item_nbr"])

# Filter data for selected combination
print(f"Training model for store {store_id} and item {item_id}")
df = df_train[(df_train["store_nbr"] == store_id) & (df_train["item_nbr"] == item_id)].copy()
df_train = df.sort_values("date")

Training model for store 24 and item 220435


## 3.4 Train/Test-Split

This section splits the dataset into training and test sets based on a predefined `CUTOFF_DATE`

In [10]:
# Train/Test Split with Cutoff from config
train_df = df_train.loc[df_train["date"] <= cfg.CUTOFF_DATE].copy()
test_df  = df_train.loc[df_train["date"] >  cfg.CUTOFF_DATE].copy()

non_numeric_cols = df_train.select_dtypes(include=["datetime64[ns]", "datetime64", "object"]).columns

X_train = train_df.drop(columns=non_numeric_cols).drop(columns=[cfg.TARGET])
y_train = np.log1p(train_df[cfg.TARGET])
X_test  = test_df.drop(columns=non_numeric_cols).drop(columns=[cfg.TARGET])
y_test  = np.log1p(test_df[cfg.TARGET])

# 4. XGBoost

## 4.1 Hyperparameter-Optimized XGBoost Model (Hyperopt)

This code performs **automated hyperparameter tuning** and training of an XGBoost model to forecast `unit_sales` for a given store and item using **Hyperopt** and **MLflow**.

### Purpose
To find the best-performing XGBoost model configuration for each store/item combination through hyperparameter optimization. The goal is to minimize forecast error using a data-driven, repeatable search strategy.

### What the code does
- Defines a **hyperparameter search space** for key XGBoost settings (e.g., `max_depth`, `learning_rate`, `n_estimators`)
- Uses **Hyperopt** to explore this space with a loss function based on `MAE`
- Trains 25 model variations (`max_evals=25`) on the training set
- Selects the best model and evaluates it on the test set
- Logs the selected model, parameters, and metrics (`MAE`, `R²`) to **MLflow**
- Saves the final model to disk for reuse or deployment

This setup supports consistent, scalable model tuning and tracking, enabling per-store/item optimization for time series forecasting tasks.

In [ ]:
# --- Imports ---
import app.config as cfg  # Optional config file (if available)

# -------------------- Setup --------------------
# Define paths
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_path = os.path.join(project_root, cfg.PREPARED_DATA_FILE)
mlruns_path = os.path.join(project_root, "mlruns", "xgb_hyperopt")
model_output_path = Path(project_root, cfg.XGB_ARCHIVE_DIR) / "xgb_hyperopt_global_model.pkl"

# Ensure necessary directories exist
os.makedirs(mlruns_path, exist_ok=True)
os.makedirs(model_output_path.parent, exist_ok=True)

# -------------------- Load & preprocess data --------------------
df = pd.read_csv(data_path, parse_dates=["date"])
df = df[df["unit_sales"] >= 0]
df = df[df["date"] <= cfg.CUTOFF_DATE]

# Define features and target
FEATURE_COLS = [
    'store_nbr', 'item_nbr', 'onpromotion', 'month',
    'unit_sales_7d_avg', 'family_code', 'lag_1', 'lag_7', 'rolling_mean_7'
]
TARGET = 'unit_sales'

# Feature matrix and target (log1p transformed)
X = df[FEATURE_COLS].astype("float32")
y = np.log1p(df[TARGET].values)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_test = X_test[FEATURE_COLS]  # Ensure proper order

# -------------------- Define hyperparameter search space --------------------
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.choice('n_estimators', range(50, 301, 50)),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
}

# -------------------- Objective function for optimization --------------------
def objective(params):
    model = xgb.XGBRegressor(random_state=42, **params)
    model.fit(X_train, y_train)
    y_pred = np.expm1(model.predict(X_test))  # Reverse log1p transform
    score = mean_absolute_error(np.expm1(y_test), y_pred)  # Evaluate in original scale
    return {'loss': score, 'status': STATUS_OK, 'model': model}

# -------------------- Run Hyperopt optimization --------------------
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=25, trials=trials)

# Retrieve best model from trials
best_model = trials.best_trial['result']['model']

# Predict and evaluate
y_pred_log = best_model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# -------------------- MLflow logging --------------------
mlflow.set_tracking_uri(f"file://{mlruns_path}")
mlflow.set_experiment("store_item_sales_forecast_hyperopt")

with mlflow.start_run():
    mlflow.log_param("model_type", "XGBoost + Hyperopt")
    mlflow.log_param("feature_cols", FEATURE_COLS)
    mlflow.log_params(best)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2_score", r2)

    # Save model to disk
    joblib.dump(best_model, model_output_path)
    mlflow.log_artifact(str(model_output_path), artifact_path="model")

    # Log model with schema signature and sample input
    signature = infer_signature(X_train, best_model.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="model_mlflow",
        signature=signature,
        input_example=X_train.head(2)
    )

    # Forecast vs. actual plot
    plt.figure(figsize=(10, 4))
    plt.plot(y_true[:300], label="Actual")
    plt.plot(y_pred[:300], label="Forecast", linestyle="--")
    plt.title("Forecast vs Actual")
    plt.xlabel("Index")
    plt.ylabel("Unit Sales")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    forecast_path = "forecast_vs_actual_hyperopt.png"
    plt.savefig(forecast_path)
    mlflow.log_artifact(forecast_path)
    plt.close()

    print(f"✅ Best MAE: {mae:.4f} | R²: {r2:.4f}")
    print(f"✅ Model saved to: {model_output_path}")

# --- End MLflow run if still active ---
if mlflow.active_run():
    mlflow.end_run()


100%|██████████| 25/25 [14:10<00:00, 34.00s/trial, best loss: 1.0784331925977066]
✅ Best MAE: 1.0784 | R²: 0.5962
✅ Modell gespeichert unter: /Users/jennypetschke/VS_Code_Projects/retail_demand_analysis/model/xgb/archive/xgb_hyperopt_global_model.pkl


## 4.2 XGBoost Regression Model (Global)

The following Code is the XGBoost regression model for predicting unit sales using engineered features, evaluate its performance, and log everything with MLflow (including metrics, model, and visualizations).

### Purpose  
The purpose of the following code is to train a **global XGBoost model** that predicts unit sales based on multiple engineered features. It aims to capture general sales patterns across all stores and items and evaluate the model’s performance using robust metrics. The full pipeline is tracked using **MLflow** for transparency and reproducibility.

### What the function does
- Loads and filters historical sales data up to a defined `CUTOFF_DATE`
- Selects relevant features and applies a log transformation to `unit_sales`
- Splits the dataset into training and testing subsets
- Trains an XGBoost regression model on the processed data
- Predicts test values and applies inverse transformation using `expm1`
- Evaluates model performance using `MAE` and `R²`
- Logs model parameters, evaluation metrics, and plots using **MLflow**
- Saves the trained model and feature metadata to disk
- Generates and logs:
  - A time series plot of unit sales over time
  - A forecast vs. actual plot to visualize prediction accuracy

This pipeline supports fast experimentation and ensures results are reproducible, interpretable, and ready for deployment.


In [ ]:
# --- End any active MLflow run ---
if mlflow.active_run():  # If the code was interrupted earlier and there's an active MLflow run
    mlflow.end_run()     # Properly end the run to avoid conflicts

# --- Feature columns used for training ---
FEATURE_COLS = [
    'store_nbr', 'item_nbr', 'onpromotion',
    'day_of_week', 'month', 'year', 'unit_sales_7d_avg',
    'family_code', 'lag_1', 'lag_7', 'rolling_mean_7'
]

# --- Load and filter dataset ---
df = pd.read_csv(os.path.join(project_root, cfg.PREPARED_DATA_FILE), parse_dates=["date"])
df = df[df['unit_sales'] >= 0]  # Remove negative sales
df = df[df['date'] <= cfg.CUTOFF_DATE]  # Only keep data up to cutoff date

# --- Define features and log-transformed target ---
X = df[FEATURE_COLS].astype("float64")
y = np.log1p(df['unit_sales'])  # Apply log1p to stabilize skewed distribution

# --- Train/test split ---
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Train XGBoost model ---
model = xgb.XGBRegressor(random_state=42)
model.fit(X_train, y_train)

# --- Make predictions and revert log1p transform ---
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # Convert back to original scale
y_true = np.expm1(y_test)

# --- Evaluate model performance ---
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

# --- Set up MLflow tracking ---
mlflow_tracking_dir_xgb_global = os.path.join(project_root, "mlruns", "xgb_global")
if not os.path.exists(mlflow_tracking_dir_xgb_global):
    os.makedirs(mlflow_tracking_dir_xgb_global, exist_ok=True)

mlflow.set_tracking_uri(f"file://{mlflow_tracking_dir_xgb_global}")
mlflow.set_experiment("global_xgb_sales_forecast")

# --- Start MLflow run ---
with mlflow.start_run():
    # Log parameters and evaluation metrics
    mlflow.log_param("model_type", "xgb_global")
    mlflow.log_param("features", FEATURE_COLS)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2_score", r2)

    # Save trained model and feature list
    out_path = Path(project_root, cfg.XGB_ARCHIVE_DIR) / cfg.GLOBAL_XGB_MODEL
    out_path.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump({"model": model, "features": FEATURE_COLS}, out_path)
    mlflow.log_artifact(str(out_path), artifact_path="model")

    print(f"✅ Global model saved to {out_path}")
    print(f"📊 MAE: {mae:.4f} | R2: {r2:.4f}")

    # --- Plot 1: Time series of original sales data ---
    import matplotlib.pyplot as plt
    df_sorted = df.sort_values("date")
    plt.figure(figsize=(10, 4))
    plt.plot(df_sorted["date"], df_sorted["unit_sales"], label="unit_sales", alpha=0.8)
    plt.title("Unit Sales over Time")
    plt.xlabel("Date")
    plt.ylabel("Unit Sales")
    plt.grid(True)
    plt.tight_layout()
    time_series_path = "unit_sales_over_time.png"
    plt.savefig(time_series_path)
    plt.close()
    mlflow.log_artifact(time_series_path)

    # --- Plot 2: Forecast vs actual (sample) ---
    plt.figure(figsize=(10, 4))
    sample_size = min(300, len(y_true))
    plt.plot(y_true[:sample_size], label="Actual", linewidth=2)
    plt.plot(y_pred[:sample_size], label="Forecast", linestyle="--")
    plt.title("Forecast vs. Actual (Test Set)")
    plt.xlabel("Sample")
    plt.ylabel("Unit Sales")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    forecast_path = "forecast_vs_actual.png"
    plt.savefig(forecast_path)
    plt.close()
    mlflow.log_artifact(forecast_path)

# --- Clean up: Ensure MLflow run is ended properly ---
if mlflow.active_run():
    mlflow.end_run()


✅ Global model saved to /Users/jennypetschke/VS_Code_Projects/retail_demand_analysis/model/xgb/archive/xgb_global.pkl
📊 MAE: 1.0521 | R2: 0.6061


# 5. LSTM

## 5.1 LSTM Sequence-to-Sequence Forecasting Model (Global)

This function trains a **global LSTM Seq2Seq model** for multi-step time series forecasting, such as predicting future `unit_sales`.



### Purpose
The purpose of the following Code is to learn general sales patterns from historical data and forecast multiple future time steps in a single forward pass. The model is trained on multiple features, like aggregated data, but also on store_nbr and item_nbr.

### What the function does
- Loads and filters time series data up to a defined `CUTOFF_DATE`
- Scales `unit_sales` using MinMaxScaler
- Prepares input/output sequences using a sliding window
- Builds a Sequence-to-Sequence LSTM model (Encoder–Decoder)
- Trains the model with callbacks (`EarlyStopping`, `ReduceLROnPlateau`)
- Logs all key parameters, metrics, and artifacts using **MLflow**
- Saves both the trained model and the scaler for future use

This setup ensures that the entire training process is tracked, reproducible, and ready for deployment or experimentation.


In [ ]:
# -- Robust Forecasting with log1p Target Transformation --
import importlib
import app.config as cfg
importlib.reload(cfg)

# --- Parameters ---
SEQ_LEN = 90  # Length of input/output sequences for the LSTM model
LSTM_UNITS = 8  # Number of LSTM units
EPOCHS = 100  # Maximum number of training epochs
BATCH_SIZE = 16  # Training batch size
MAX_SAMPLES = 800  # Max number of samples used for training to reduce memory usage
dtype = "float32"  # Float type for model inputs
FEATURES = ['onpromotion', 'lag_1', 'rolling_mean_7']  # Feature columns

# --- Paths ---
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_path = os.path.join(project_root, cfg.PREPARED_DATA_FILE)
mlflow_tracking_dir = os.path.join(project_root, cfg.MLFLOW_LSTM_SEQ)
model_path = Path(project_root) / cfg.LSTM_ARCHIVE_DIR / cfg.LSTM_GLOBAL_MODEL
scaler_path = Path(project_root) / cfg.SCALER_ARCHIVE_DIR / cfg.LSTM_GLOBAL_SCALER

# --- Load data ---
df = pd.read_csv(data_path, usecols=FEATURES + ['unit_sales', 'date'], parse_dates=["date"])
df = df[(df["unit_sales"] >= 0) & (df["date"] <= cfg.CUTOFF_DATE)].sort_values("date").reset_index(drop=True)
df["target"] = np.log1p(df["unit_sales"])  # Apply log1p transform to stabilize variance

# --- Feature Scaling ---
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = feature_scaler.fit_transform(df[FEATURES]).astype(dtype)
y_scaled = target_scaler.fit_transform(df[["target"]]).astype(dtype)

# --- Prepare sequence data for encoder-decoder LSTM ---
def make_seq2seq(X, y, n_in, n_out):
    X_seq, y_seq = [], []
    for i in range(len(X) - n_in - n_out + 1):
        X_seq.append(X[i:i+n_in])
        y_seq.append(y[i+n_in:i+n_in+n_out, 0])
    return np.array(X_seq), np.array(y_seq)

X_raw, y_raw = make_seq2seq(X_scaled, y_scaled, SEQ_LEN, SEQ_LEN)

X_enc = X_raw.astype(dtype)  # Encoder input
X_dec = np.zeros((len(X_raw), SEQ_LEN, 1), dtype=dtype)  # Decoder input
X_dec[:, 1:, 0] = y_raw[:, :-1]  # Shifted target as decoder input
X_dec[:, 0, 0] = X_enc[:, -1, 0]  # First input: last feature from encoder

y_seq = y_raw.reshape(len(y_raw), SEQ_LEN, 1).astype(dtype)  # Decoder output

# Reduce sample size if necessary
if len(X_enc) > MAX_SAMPLES:
    X_enc = X_enc[-MAX_SAMPLES:]
    X_dec = X_dec[-MAX_SAMPLES:]
    y_seq = y_seq[-MAX_SAMPLES:]

# --- Build LSTM Seq2Seq model ---
input_dim = X_enc.shape[2]

# Encoder
enc_inputs = Input(shape=(SEQ_LEN, input_dim), name="encoder_input")
_, state_h, state_c = LSTM(LSTM_UNITS, return_state=True)(enc_inputs)

# Decoder
dec_inputs = Input(shape=(SEQ_LEN, 1), name="decoder_input")
dec_outputs = LSTM(LSTM_UNITS, return_sequences=True)(dec_inputs, initial_state=[state_h, state_c])
outputs = Dense(1)(dec_outputs)

# Compile model
model = Model([enc_inputs, dec_inputs], outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss="mae")

# --- MLflow setup ---
mlflow.set_tracking_uri(f"file://{mlflow_tracking_dir}")
mlflow.set_experiment("lstm_seq2seq_log1p")

if mlflow.active_run():
    mlflow.end_run()  # End any active run

with mlflow.start_run():
    # Log model parameters
    mlflow.log_params({
        "seq_len": SEQ_LEN,
        "lstm_units": LSTM_UNITS,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "features": FEATURES
    })

    # Train the model
    history = model.fit([X_enc, X_dec], y_seq, epochs=EPOCHS, batch_size=BATCH_SIZE,
                        callbacks=[
                            EarlyStopping(monitor="loss", patience=3, restore_best_weights=True),
                            ReduceLROnPlateau(monitor="loss", factor=0.5, patience=2)
                        ],
                        verbose=1)
    
    # Log final training loss
    mlflow.log_metric("final_loss", float(history.history["loss"][-1]))

    # --- Forecasting ---
    X_pred_enc = X_enc[-1:]  # Last known sequence for encoder
    X_pred_dec = np.zeros((1, SEQ_LEN, 1), dtype=dtype)

    # Seed decoder with the last known log1p target value
    last_log_target = target_scaler.transform([[np.log1p(df["unit_sales"].iloc[-1])]])[0, 0]
    X_pred_dec[:, 0, 0] = last_log_target

    # Empty forecast array
    forecast_scaled = np.zeros((SEQ_LEN, 1), dtype=dtype)

    # Auto-regressive forecasting
    for t in range(SEQ_LEN):
        pred_step = model.predict([X_pred_enc, X_pred_dec], verbose=0)[0]
        forecast_scaled[t, 0] = pred_step[t, 0]
        if t + 1 < SEQ_LEN:
            X_pred_dec[0, t + 1, 0] = forecast_scaled[t, 0]

    # Inverse transform and convert back from log scale
    forecast_log = target_scaler.inverse_transform(forecast_scaled)
    forecast = np.expm1(forecast_log).flatten()

    # --- Plot forecast ---
    future_dates = pd.date_range(df["date"].max() + pd.Timedelta(days=1), periods=SEQ_LEN)
    plt.figure(figsize=(10, 4))
    plt.plot(future_dates, forecast, label="Forecast", linewidth=2, linestyle="--")
    plt.title("LSTM Seq2Seq Forecast (90 Days)")
    plt.xlabel("Date")
    plt.ylabel("Unit Sales")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Save and log forecast plot
    plot_path = "forecast_log1p_plot.png"
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    plt.close()

    # --- Save model and scaler ---
    model_path.parent.mkdir(parents=True, exist_ok=True)
    scaler_path.parent.mkdir(parents=True, exist_ok=True)
    model.save(model_path)
    joblib.dump(target_scaler, scaler_path)

    # Log files to MLflow
    mlflow.log_artifact(str(model_path))
    mlflow.log_artifact(str(scaler_path))

    print("✅ Model and forecast successfully saved.")


ModuleNotFoundError: No module named 'app'